## Calculate All Statistics
Here's the idea
- This notebook is made to be able to run all of our statistics very quickly (well not quickly but at least with one button push rather than hundreds) across all sample sizes, and runs within that
- End goal of this notebook is to get a table where all 6 statistics are computed in one output per one sample size - for a more detailed look into each of the statistics and what goes into them you can look into the individual notebooks
- This is going to be fairly nasty looking but we did this so we can press one button and get all the statistics all at once

In [20]:
import pandas as pd
import networkx as nx # documentation at https://networkx.org/documentation/
import matplotlib.pyplot as plt
from scipy import stats
import collections
import numpy as np

In [7]:
data = pd.read_csv("./Data/web-Stanford.txt", sep="\t")
in_degree_OG = data.groupby("ToNode").count()["FromNode"] #for in degree statistic
out_degree_OG = data.groupby("FromNode").count()["ToNode"] #for out degree statistic
G_Original = nx.DiGraph()
G_Original = nx.from_pandas_edgelist(data, "FromNode", "ToNode", create_using=nx.DiGraph())
wcc_OG = nx.weakly_connected_components(G_Original)
wcc_sizes_OG = collections.Counter([len(wcc) for wcc in wcc_OG]).keys()
scc_OG = nx.strongly_connected_components(G_Original)
scc_sizes_OG = collections.Counter([len(wcc) for wcc in scc_OG]).keys()
cc_OG = nx.average_clustering(G_Original)

In [25]:
cc_OG = nx.clustering(G_Original)

In [ ]:
list_of_nodes = np.unique(np.array(data["FromNode"]))
edge_holder = {}
for nodes in list_of_nodes:
    outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
    edge_holder[nodes] = set(outlinks)

In [24]:
hop_plot_3_OG = counting(3)

In [26]:
largestcc = max(nx.weakly_connected_components(G_Original), key=len)
edge_holder = {}
for nodes in list_of_nodes:
    if(nodes not in largestcc):
        continue
    outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
    filtered = list(filter(lambda x: x in largestcc, outlinks))
    if(len(filtered) > 0):
        edge_holder[nodes] = set(filtered)

hop_plot_3_wcc_OG = counting(3)

In [9]:
def indegree():
    in_degree_sample = input_data.groupby("ToNode").count()["FromNode"]
    return stats.ks_2samp(in_degree_OG, in_degree_sample)[0] #returns the d-statistic

In [10]:
def outdegree():
    out_degree_sample = input_data.groupby("FromNode").count()["ToNode"]
    return stats.ks_2samp(out_degree_OG, out_degree_sample)[0] #returns the d-statistic

In [12]:
def WCC():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    wcc_sample = nx.weakly_connected_components(G_sample)
    wcc_sizes_sample = collections.Counter([len(wcc) for wcc in wcc_sample]).keys()
    return stats.ks_2samp(list(wcc_sizes_OG), list(wcc_sizes_sample))[0]

In [13]:
def SCC():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    scc_sample = nx.strongly_connected_components(G_sample)
    scc_sizes_sample = collections.Counter([len(wcc) for wcc in scc_sample]).keys()
    return stats.ks_2samp(list(scc_sizes_OG), list(scc_sizes_sample))[0]

In [14]:
def clusteringCoef():
    G_sample = nx.DiGraph()
    G_sample = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    cc_sample = nx.clustering(G_sample)
    return stats.ks_2samp(list(cc_OG), list(cc_sample))[0]

In [15]:
def hopping(h, edge_holder): #now this returns all edges that can be reached in some amount of hops
    #this works for two steps now need to make it work for 3,4, etc...
    #first step is for each h create a new "edge_holder" that holds the elements it's connected ot in h steps
    edge_holder2 = edge_holder.copy()
    for nodes in edge_holder:
        last_nodes = edge_holder[nodes]
        hops = h
        while(hops > 1):
            temp = set()
            for outlinks in last_nodes:
                if(outlinks in edge_holder):
                    temp = set.union(temp, edge_holder[outlinks])            
            edge_holder2[nodes] = set.union(temp, edge_holder2[nodes])
            hops = hops - 1
            last_nodes = temp
    
    
            
    return edge_holder2

def counting(num_steps, edge_holder): #what actually needs to be run for hop plot to run
    e = hopping(num_steps, edge_holder);
    pairs = 0;
    for nodes in e:
        for outlinks in e[nodes]:
            if(outlinks in e and nodes in e[outlinks]):
                pairs += 1;
    
    return int(pairs/2); #since we end up double counting

In [16]:
def edge_holder_normal():
    list_of_nodes = np.unique(np.array(input_data["FromNode"]))
    edge_holder = {}
    for nodes in list_of_nodes:
        outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
        edge_holder[nodes] = set(outlinks)
    return edge_holder

In [17]:
def edge_holder_wcc():
    G_smp = nx.DiGraph()
    G_smp = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
    largestcc = max(nx.weakly_connected_components(G_smp), key=len)
    edge_holder = {}
    for nodes in list_of_nodes:
        if(nodes not in largestcc):
            continue
        outlinks = list(input_data[input_data["FromNode"] == nodes]["ToNode"])
        filtered = list(filter(lambda x: x in largestcc, outlinks))
        if(len(filtered) > 0):
            edge_holder[nodes] = set(filtered)
    return edge_holder

In [18]:
ssizes = ["5%", "10%", "15%", "20%"]
runnumbers = ["1", "2", "3", "4", "5"]
typeofsamples = ["FF", "RNE", "RW", "RN"]

In [3]:
dic = {"RW5%" : [], "RNE5%": [], "FF5%": [], "RN5%": [],
       "RW10%" : [], "RNE10%": [], "FF10%": [], "RN10%": [], 
      "RW15%" : [], "RNE15%": [], "FF15%": [], "RN15%": [], 
       "RW20%" : [], "RNE20%": [], "FF20%": [], "RN20%": [], 
      }

In [19]:
dic2 = {"RW5%" : [], "RNE5%": [], "FF5%": [], "RN5%": [],
       "RW10%" : [], "RNE10%": [], "FF10%": [], "RN10%": [], 
      "RW15%" : [], "RNE15%": [], "FF15%": [], "RN15%": [], 
       "RW20%" : [], "RNE20%": [], "FF20%": [], "RN20%": [], 
      }

In [24]:
for ssize in ssizes:
    for typeofsample in typeofsamples:
        keyname = typeofsample + ssize #keeping track of where to store things
        ind = []
        outd = []
        wcc = []
        scc = []
        hopnormal = []
        hopwcc = []
        cc = []
        for runnumber in runnumbers:
            
            name = "./Samples/" + ssize + "/" + typeofsample + "/" + "Output" + typeofsample + ssize + runnumber + ".csv"
            input_data = pd.read_csv(name);
#             ind.append(indegree())
#             outd.append(outdegree())
#             wcc.append(WCC())
#             scc.append(SCC())
#             eh = edge_holder_normal()
#             hopnormal.append(counting(3, eh))
#             eh2 = edge_holder_wcc()
#             hopwcc.append(counting(3, eh))
#             cc.append(clusteringCoef())
    
    
#         dic[keyname].append(ind)
#         dic[keyname].append(outd)
#         dic[keyname].append(wcc)
#         dic[keyname].append(scc)
#         dic[keyname].append(hopnormal)
#         dic[keyname].append(hopwcc)
#         dic[keyname].append(cc)
            
            
#         print(dic)
# op = pd.DataFrame.from_dict(dic)
# op.to_csv("Finally?.csv")

./Samples/5%/FF/OutputFF5%1.csv
./Samples/5%/FF/OutputFF5%2.csv
./Samples/5%/FF/OutputFF5%3.csv
./Samples/5%/FF/OutputFF5%4.csv
./Samples/5%/FF/OutputFF5%5.csv
./Samples/5%/RNE/OutputRNE5%1.csv
./Samples/5%/RNE/OutputRNE5%2.csv
./Samples/5%/RNE/OutputRNE5%3.csv
./Samples/5%/RNE/OutputRNE5%4.csv
./Samples/5%/RNE/OutputRNE5%5.csv
./Samples/5%/RW/OutputRW5%1.csv
./Samples/5%/RW/OutputRW5%2.csv
./Samples/5%/RW/OutputRW5%3.csv
./Samples/5%/RW/OutputRW5%4.csv
./Samples/5%/RW/OutputRW5%5.csv
./Samples/5%/RN/OutputRN5%1.csv
./Samples/5%/RN/OutputRN5%2.csv
./Samples/5%/RN/OutputRN5%3.csv
./Samples/5%/RN/OutputRN5%4.csv
./Samples/5%/RN/OutputRN5%5.csv
./Samples/10%/FF/OutputFF10%1.csv
./Samples/10%/FF/OutputFF10%2.csv
./Samples/10%/FF/OutputFF10%3.csv
./Samples/10%/FF/OutputFF10%4.csv
./Samples/10%/FF/OutputFF10%5.csv
./Samples/10%/RNE/OutputRNE10%1.csv
./Samples/10%/RNE/OutputRNE10%2.csv
./Samples/10%/RNE/OutputRNE10%3.csv
./Samples/10%/RNE/OutputRNE10%4.csv
./Samples/10%/RNE/OutputRNE10%5.csv


## End